In [1]:
import pandas as pd
import numpy as np

## Read in DOT df with avg speeds

In [2]:
df = pd.read_csv("dot_spd_comb.csv", index_col= False)

In [3]:
df

,road_name,avg_spd
0,Farrington Highway,33.640771
1,Joseph P Leong Highway,43.177286
2,Kamehameha Highway,35.413415
3,Front Street,20.696062
4,Kapoho Kalapana Road,30.846234
...,...,...
626,Kao Road,35.000000
627,H-1_WB_7,55.000000
628,Saratoga Road,45.000000
629,Ala Luina Street (Port of Kahului),7.010062


In [4]:
cmn_name = ["Exit", "Ramp"]

In [5]:
mask = df["road_name"].apply(lambda x: any(item for item in cmn_name if item in x))

In [6]:
df1 = df[mask]
df1

,road_name,avg_spd


## Prep Hnl Road network for join
Filter out only those roads that appear in the DOT dataset with avg speeds

Initial total number of rows in State GIS network dataset= 27,064

segment_id column is the unique identifier of each road

In [7]:
df_state = pd.read_excel("hnl_roads.xls")

/var/folders/7z/bh_vxcz94c17ntk9rz3rrq3h0000gn/T/ipykernel_10909/1911228405.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  df_state = pd.read_excel("hnl_roads.xls")


In [8]:
# Number of roads with no name (806)

df_state[df_state["fullname"].isna()]

,objectid,segmentid,fullname,class,street_cla,zipcoder,zipcodel,townl,townr,neighborr,oneway,edittype,revisedate,owner,loaddate,st_lengths,speed
1,2,252357,NaN,A41,4,96797.0,96797.0,Waipahu,Waipahu,WAIPIO PENINSULA,0,NaN,2003-06-23,FED,2009-05-15,139.552979,0
6,7,766,NaN,A41,1,96791.0,96791.0,Waialua,Waialua,NORTH SHORE,0,NaN,2003-05-21,UNKNOWN,2009-05-15,223.959035,0
43,44,1256,NaN,A41,1,96792.0,96792.0,Waianae,Waianae,WAIANAE,0,NaN,2003-05-21,UNKNOWN,2009-05-15,96.701214,0
59,60,400091,NaN,A41,4,96819.0,96819.0,Honolulu / Tripler,Honolulu / Tripler,KAPALAMA MIL RES,0,NaN,2003-06-23,FED,2009-05-15,12.064430,0
93,94,251981,NaN,A41,4,96792.0,96792.0,Waianae,Waianae,LUALUALEI,0,NaN,2003-06-23,FED,2009-05-15,279.160741,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24833,24834,250100,NaN,A41,4,96860.0,96860.0,Honolulu / Halsey Ter,Honolulu / Halsey Ter,HALE MOKU,0,NaN,2015-03-06,FED,2015-03-06,35.590591,0
24885,24886,243,NaN,A41,1,96731.0,96731.0,Laie,Laie,KOOLAULOA,0,NaN,2015-03-06,UNKNOWN,2015-03-06,440.696441,0
24908,24909,350563,NaN,A41,4,96744.0,96744.0,Kailua,Kailua,MCBH,0,0.0,2015-03-06,FED,2015-03-06,135.068428,0
24933,24934,251414,NaN,A41,4,96860.0,96860.0,Honolulu / Halsey Ter,Honolulu / Halsey Ter,PEARL HARBOR,0,NaN,2015-03-06,FED,2015-03-06,112.992943,0


In [9]:
# Capitalise first letter of road name

df_state["fullname"] = df_state["fullname"].str.title()

In [10]:
# Number of unique road names (8097)

len(list(df_state["fullname"].unique()))

8097

In [11]:
# Replace abbreviated names with longform

df_state["fullname"] = df_state["fullname"].str.replace("Hwy", "Highway")
df_state["fullname"] = df_state["fullname"].str.replace("St", "Street")

### Interim Validation check on specific roads

In [12]:
# Drop na values before able to validate

df_notna = df_state[~df_state["fullname"].isna()]
df_notna[df_notna["fullname"].str.contains('Street')]

,objectid,segmentid,fullname,class,street_cla,zipcoder,zipcodel,townl,townr,neighborr,oneway,edittype,revisedate,owner,loaddate,st_lengths,speed
0,1,34403,Hoonana Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,EWA,0,NaN,2004-12-08,CITY,2009-05-15,59.397475,0
2,3,33706,Maikoiko Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,WAIPAHU,0,NaN,2003-05-21,UNKNOWN,2009-05-15,108.801348,0
4,5,19423,Lusitana Street,A41,4,96813.0,96813.0,Honolulu / Downtown,Honolulu / Downtown,DOWNTOWN,2,NaN,2007-04-13,CITY,2009-05-15,47.902809,0
5,6,692,Pikai Street,A41,4,96712.0,96712.0,Haleiwa,Haleiwa,NORTH SHORE,0,NaN,2003-05-21,CITY,2009-05-15,134.519100,0
7,8,19281,Kealahou Street,A41,4,96825.0,96825.0,Honolulu / Hawaii Kai,Honolulu / Hawaii Kai,HAWAII KAI,0,NaN,2003-05-21,CITY,2009-05-15,51.109878,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27059,27060,44346,Puepue Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2020-09-08,PRIVATE,2020-09-08,24.268751,0
27060,27061,44345,Puepue Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2020-09-08,PRIVATE,2020-09-08,93.879791,0
27061,27062,44083,Hulukupuna Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,Mililani-Waipio-Melemanu,0,4.0,2020-10-13,CITY,2020-10-13,172.651129,0
27062,27063,44445,Hulukupuna Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,Mililani-Waipio-Melemanu,0,0.0,2020-10-13,CITY,2020-10-13,46.673790,0


### Join avg speeds in DOT to State dataset

Create list of 631 unique road names in DOT dataset

Try to mask road names in State GIS data, see how much overlap there is 

In [13]:
cmn_name = list(df["road_name"].unique())

In [14]:
# These are the road names in the DOT dataset with avg speeds

cmn_name

['Farrington Highway',
 'Joseph P Leong Highway',
 'Kamehameha Highway',
 'Front Street',
 'Kapoho Kalapana Road',
 'Lualualei Homestead Road',
 'Halemaumau Street',
 'Pahoa-Kalapana Road',
 'Pahoa Bypass Road',
 'Waha Road',
 'Isenberg Street',
 'Hawaii Belt Road',
 'Kapahulu Avenue',
 'Palani Road',
 'Farrington Highway-Kunia Road to H-1',
 'ramp to Kamehameha Highway',
 'Honoapiilani Highway',
 'Kalihi Street',
 'Hui Iwa Street',
 'Honoapiilani Highway Realignment',
 'Makawao Avenue',
 'Likelike Highway',
 'Mamalahoa Highway',
 'Dillingham Boulevard',
 'Haleakala Crater Road',
 'Kinoole Street',
 'Iolani Street',
 'Laukona Street',
 'Kuhio Avenue',
 'West Puainako Street',
 'Volcano Road',
 'Hawi Road',
 'Kohala Mountain Road',
 'Koloa Road',
 'Poipu Road',
 'Kalanianaole Highway',
 'Saddle Road',
 'Kamehameha V Highway',
 'Likini Street',
 'Kahekili Highway',
 'Pahoa-Kapoho Road',
 'Old Keaau-Pahoa Road',
 'Keeau-Pahoa Road',
 'Ke-ala-o-Keawe Road',
 'Makani Road',
 'Luluku Road',


In [15]:
df_state = df_state.rename(columns={"fullname": "road_name"})
df_state[df_state["road_name"].isin(cmn_name)]

,objectid,segmentid,road_name,class,street_cla,zipcoder,zipcodel,townl,townr,neighborr,oneway,edittype,revisedate,owner,loaddate,st_lengths,speed
4,5,19423,Lusitana Street,A41,4,96813.0,96813.0,Honolulu / Downtown,Honolulu / Downtown,DOWNTOWN,2,NaN,2007-04-13,CITY,2009-05-15,47.902809,0
18,19,7946,Paiwa Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,WAIPAHU,0,4.0,2009-06-17,CITY,2009-06-17,94.676244,0
26,27,250980,Main Street,A41,4,96818.0,96818.0,Honolulu / Halsey Ter,Honolulu / Halsey Ter,HALE MOKU,0,NaN,2006-12-21,FED,2009-05-15,116.060398,0
36,37,20478,Pensacola Street,A41,4,96814.0,96814.0,Honolulu / Downtown,Honolulu / Downtown,ALA MOANA/KAKAAKO,2,NaN,2003-05-21,CITY,2009-05-15,121.044852,0
37,38,8882,Farrington Highway,A21,8,96792.0,96792.0,Waianae,Waianae,WAIANAE,0,NaN,2003-05-21,STATE,2009-05-15,440.395735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26943,26944,42812,Kulanihakoi Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,4.0,2019-02-05,City,2019-02-05,22.361565,0
26956,26957,43102,Kulanihakoi Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2019-03-20,CITY,2019-03-20,197.856348,0
26957,26958,43103,Kulanihakoi Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2019-03-20,CITY,2019-03-20,222.698622,0
26958,26959,43104,Kulanihakoi Street,A41,4,96706.0,96706.0,Kapolei,Kapolei,Ewa,0,0.0,2019-03-20,CITY,2019-03-20,256.360620,0


### Try to trasfer avg speed fro DOT to State dataset 
For the road names that match both datasets

In [16]:
df_state1 = df_state.merge(df, on = "road_name", how = "left")

In [17]:
# No. of State dataset roads still missing avg_speeds

df_state1["avg_spd"].isna().sum()

25055

In [18]:
df_state1 = df_state1.drop(['speed'], axis =1)

In [19]:
df_state1

,objectid,segmentid,road_name,class,street_cla,zipcoder,zipcodel,townl,townr,neighborr,oneway,edittype,revisedate,owner,loaddate,st_lengths,avg_spd
0,1,34403,Hoonana Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,EWA,0,NaN,2004-12-08,CITY,2009-05-15,59.397475,NaN
1,2,252357,NaN,A41,4,96797.0,96797.0,Waipahu,Waipahu,WAIPIO PENINSULA,0,NaN,2003-06-23,FED,2009-05-15,139.552979,NaN
2,3,33706,Maikoiko Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,WAIPAHU,0,NaN,2003-05-21,UNKNOWN,2009-05-15,108.801348,NaN
3,4,251245,Lexington Blvd,A41,4,96818.0,96818.0,Honolulu / Halsey Ter,Honolulu / Halsey Ter,FORD ISLAND,0,NaN,2003-06-23,FED,2009-05-15,134.637306,NaN
4,5,19423,Lusitana Street,A41,4,96813.0,96813.0,Honolulu / Downtown,Honolulu / Downtown,DOWNTOWN,2,NaN,2007-04-13,CITY,2009-05-15,47.902809,27.969604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27059,27060,44346,Puepue Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2020-09-08,PRIVATE,2020-09-08,24.268751,NaN
27060,27061,44345,Puepue Street,A41,4,96706.0,96706.0,Ewa Beach,Ewa Beach,Ewa,0,0.0,2020-09-08,PRIVATE,2020-09-08,93.879791,NaN
27061,27062,44083,Hulukupuna Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,Mililani-Waipio-Melemanu,0,4.0,2020-10-13,CITY,2020-10-13,172.651129,NaN
27062,27063,44445,Hulukupuna Street,A41,4,96797.0,96797.0,Waipahu,Waipahu,Mililani-Waipio-Melemanu,0,0.0,2020-10-13,CITY,2020-10-13,46.673790,NaN


In [60]:
df_state1.to_csv("final_v1.csv", index= False)